# Extract coordinates and keywords from Wikipedia

Wikipedia allows you to download the entirety of its contents. For English only (and without talk pages) that comes out to about 16 GB of data in the zipped format. We're only interested in articles with coordinates specified for the article itself (rather than something the article refers to). So we need to 1) be able to read in a page, given either a page ID or the page's title, and 2) query all pages with coordinates with a certain display type.

Wikipedia provides an index file to go with the database dump, that lets you read articles in batches of 100 using byte locations. But I'd rather have the exact index to all the pages; that will save time when we're iterating over a subset of the pages later. And compared to other things I'm doing, building my own index isn't terribly computationally expensive.

First, let's tell it where the dump file is. I made a 10 MB sample file that we can use.

In [1]:
from pathlib import Path
from wikiparse import config

xml_filename = config.xml
scratch_folder = Path(config.folder)

I could save lots of hard drive space and some code if I used the bzipped XML file provided by Wikipedia along with their index. But the computationally intensive part isn't indexing the XML dump, it's the page tokenization and clustering which take forever. I'll leave it as an exercise for the reader whether switching this part over to using the bzip dump is better.

## What we want to create

For each page, we're going to have the start and end indices, the page number, title, and the coord tag. We'll ignore any pages that don't have the string `Coord` or `coord`.

Here are the databases we're creating:
 - `index`: maps page number to the title, start and end indices, and raw coord tag
 - `coords`: maps page number to lots of attributes in a coord tag
 - `title`: maps page title to page number
 
Here's an example of a coord tag from a Wikipedia article:
```
coord|42.440556|-98.148083|type:landmark|name=Ashfall Fossil Beds
```

Let's get started.

In [2]:
from wikiparse import geo_indexer
import time
pipeline_start = time.time()

In [3]:
indexer = geo_indexer.Indexer(xml_filename,
            scratch_folder=scratch_folder)
indexer.load()

opening C:\Users\rowan\Documents\geowiki\scratch\index.db
Ready. Metadata: []
iterating 100.0% of pages took 73.28 minutes53k k 
7.67 % contained coordinates tag
Creating title dictionary
UNIQUE constraint failed: titles.title KBZZ (AM)


How many pages do we have? This is the number that have coord tags.

In [4]:
len(indexer.get_page_numbers())

647725

Let's check out a page.

In [5]:
page = indexer.get_page_by_num(indexer.get_page_numbers()[5])
print(f'"{page.title}" \n\n{page.text[:1000]}')

"Anatolia" 





 its eastern and southeastern borders are widely taken to be Turkeys eastern border. By some definitions, the Armenian Highlands lies beyond the boundary of the Anatolian plateau. The official name of this inland region is the Eastern Anatolia Region.

The Ancient Anatolians ancient inhabitants of Anatolia spoke the now-extinct Anatolian languages, which were largely replaced by the Greek language starting from classical antiquity and during the Hellenistic period Hellenistic, Roman Empire Roman and Byzantine Empire Byzantine periods. Major Anatolian languages included Hittite language Hittite, Luwian language Luwian, and Lydian language Lydian, among other more poorly attested relatives. The Turkification of Anatolia began under the Seljuk Empire in the late 11th century and continued under the Ottoman Empire between the late 13th and early 20th centuries. However, various non-Turkic languages continue to be spoken by minorities in Anatolia today, including Kurdish la

We're losing _some_ of the actual text of the Wikipedia article, but generally the filter's doing a good job of keeping out the metadata and keeping in the text.

In [6]:
took = time.time() - pipeline_start
if took < 60:
    print("pipeline took", round(took, 2), "seconds")
elif took < 3600:
    print("pipeline took", round(took/60, 2), "minutes")
else:
    print("pipeline took", round(took/60/60, 2), "hours")

pipeline took 2.01 hours
